In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ML_df.csv')
df.head()

,carat,depth,table,x,y,z,Good,Ideal,Premium,Very Good,...,I,J,IF,SI1,SI2,VS1,VS2,VVS1,VVS2,price
0,-1.198168,-0.174092,-1.099672,-1.587837,-1.536196,-1.571129,0,1,0,0,...,0,0,0,0,1,0,0,0,0,326
1,-1.240361,-1.360738,1.585529,-1.641325,-1.658774,-1.741175,0,0,1,0,...,0,0,0,1,0,0,0,0,0,326
2,-1.198168,-3.385019,3.375663,-1.498691,-1.457395,-1.741175,1,0,0,0,...,0,0,0,0,0,1,0,0,0,327
3,-1.071587,0.454133,0.242928,-1.364971,-1.317305,-1.287720,0,0,1,0,...,1,0,0,0,0,0,1,0,0,334
4,-1.029394,1.082358,0.242928,-1.240167,-1.212238,-1.117674,1,0,0,0,...,0,1,0,0,1,0,0,0,0,335


In [3]:
X = df.drop('price',axis=1)
y = df.price

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
train,val_train,test,val_test = train_test_split(X,y,test_size=0.5,random_state=33)

In [7]:
x_train,x_test,y_train,y_test = train_test_split(train,test,test_size=0.2)

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

#### KNN

In [10]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)

KNeighborsRegressor()

In [11]:
knn.score(x_test,y_test)

0.9493504426473592

In [12]:
param_grid ={
    'n_neighbors':[3,5,7,9],
    'weights':['uniform','distance'],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
}

In [13]:
from sklearn.model_selection import RandomizedSearchCV

In [14]:
rand_search_knn = RandomizedSearchCV(estimator=knn,param_distributions=param_grid,cv=5,n_jobs=-1,verbose=3,random_state=33)
rand_search_knn.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'n_neighbors': [3, 5, 7, 9],
                                        'weights': ['uniform', 'distance']},
                   random_state=33, verbose=3)

In [15]:
rand_search_knn.best_params_

{'weights': 'distance', 'n_neighbors': 7, 'algorithm': 'kd_tree'}

In [16]:
best_grid_knn = rand_search_knn.best_estimator_

### RandomForest

In [17]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)

RandomForestRegressor()

In [18]:
rf.score(x_train,y_train)

0.9960868255325329

In [19]:
rf.score(x_test,y_test)

0.9704611921701249

In [21]:
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = range(2,11,1)
# Minimum number of samples required at each leaf node
min_samples_leaf = range(2,11,1)
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [22]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [25]:
rand_search_rf = RandomizedSearchCV(estimator=rf,param_distributions=random_grid,cv=5,n_jobs=-1,verbose=3,random_state=33)
rand_search_rf.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/ronak/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': range(2, 11),
                                        'min_samples_split': range(2, 11),
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=33, verbose=3)

In [26]:
rand_search_rf.best_params_

{'n_estimators': 1400,
 'min_samples_split': 3,
 'min_samples_leaf': 5,
 'max_features': 'auto',
 'max_depth': 110,
 'bootstrap': True}

In [28]:
best_grid_rf = rand_search_rf.best_estimator_

In [29]:
predict_val1 = best_grid_knn.predict(val_train)
predict_val2 = best_grid_rf.predict(val_train)

In [30]:
predict_val = np.column_stack((predict_val1,predict_val2))

In [31]:
predict_test1 = best_grid_knn.predict(x_test)
predict_test2 = best_grid_rf.predict(x_test)

In [32]:
predict_test = np.column_stack((predict_test1,predict_test2))

### xgboost

In [33]:
xgb = XGBRegressor()
xgb.fit(predict_val,val_test)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
xgb.score(predict_test,y_test)

0.9640065638831322

In [35]:
param_grid = {
    'booster':['gbtree','gblinear'],
    'eta':[0.01,0.1,0.3,0.5,1],
    'min_child_weight': range(1,5,1),
    'max_depth': range(2,10,1),
    'eval_metric':['rmse','mae','logloss','error','mlogloss','auc']
}

In [37]:
rand_search_xgb = RandomizedSearchCV(estimator=xgb,param_distributions=param_grid,n_iter=100,cv=5,verbose=2,n_jobs=-1,random_state=33)
rand_search_xgb.fit(predict_val,val_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          gpu_id=-1, importance_type='gain',
                                          interaction_constraints='',
                                          learning_rate=0.300000012,
                                          max_delta_step=0, max_depth=6,
                                          min_child_weight=1, missing=nan,
                                          monotone_constraints='()',
                                          n_estimators=100, n_jobs=4,
                                          num_par...
                                          reg_alpha=0, reg_lambda=1,
                                          scale_pos_weight=1, subsample=1,
   

In [38]:
rand_search_xgb.best_params_

{'min_child_weight': 1,
 'max_depth': 2,
 'eval_metric': 'mae',
 'eta': 0.1,
 'booster': 'gbtree'}

In [39]:
best_rand_xgb = rand_search_xgb.best_estimator_

In [40]:
best_rand_xgb.score(predict_val,val_test)

0.9702106182763943